Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 7.91 ms, sys: 417 µs, total: 8.33 ms
Wall time: 11.5 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

<Client: 'inproc://10.1.0.177/8031/1' processes=1 threads=6, memory=6.78 GiB>

1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 839 ms, sys: 58.5 ms, total: 897 ms
Wall time: 2.04 s


[1.8252550365561842,
 1.5550545936035625,
 1.6907276234873512,
 1.6202717625203413,
 1.8835811185489064,
 1.2862821473283894,
 1.7773346848668286,
 1.8306617675814014,
 1.130470283127622,
 1.6313873219580604,
 1.7540647281272639,
 1.7547546795844906,
 1.6157122325167903,
 1.8660950649528258,
 1.8692319460146856,
 1.114112415773292,
 1.6885755289030229,
 1.1703107527042287,
 1.303386566229456,
 1.3039751344412571,
 1.6943885707599082,
 1.244375135378653,
 1.5870503367840068,
 1.3801208298878889,
 1.3189825848334102,
 1.2967692903796697,
 1.984394022462984,
 1.3409638623068592,
 1.6084309314787837,
 1.9265048793325303,
 1.1137901946762168,
 1.3607313058470525,
 1.4395297894456272,
 1.2775964567904536,
 1.8941722500415992,
 1.2449427338156955,
 1.0681724934527757,
 1.7638443951313765,
 1.3940990175935046,
 1.5547184529974205,
 1.4463502841776998,
 1.7015358219644472,
 1.5750456968149102,
 1.7925397382063857,
 1.8774645103607037,
 1.8936552078542839,
 1.4247712098402847,
 1.962006113528477

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.0527748327605728,
 1.2586594262148694,
 1.7497059135891613,
 1.7365793035824366,
 1.481658710134257,
 1.1326925545264896,
 1.5009930424376676,
 1.499090353011983,
 1.7515669445403932,
 1.0496395885320138,
 1.5204750919654206,
 1.200074694962656,
 1.437790743061937,
 1.7021013542211318,
 1.078461941003887,
 1.3246807762862165,
 1.1379884844496433,
 1.8742831841662992,
 1.3811893866805982,
 1.5396064308424602,
 1.200853594084444,
 1.7903026406395437,
 1.3856873104706842,
 1.1808910830495458,
 1.4181145159630875,
 1.6608209891648733,
 1.8299810285777818,
 1.6556267398010733,
 1.990789896696033,
 1.9838851776732658,
 1.9735385346664365,
 1.1858014662072716,
 1.285700430604201,
 1.1535763729774169,
 1.7447023621611806,
 1.8153549013506878,
 1.4917868802480148,
 1.9134811927909974,
 1.1260262936086436,
 1.4463004144404348,
 1.3494350334557823,
 1.6656314538656445,
 1.2479175497418122,
 1.844539652938102,
 1.4563225324203646,
 1.290591712331262,
 1.6513364640939046,
 1.1952826128421954,
 1

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 358 ms, sys: 39.4 ms, total: 397 ms
Wall time: 2.58 s
